### Pre-Requisites

In [ ]:
#!pip install folium
#!pip install gpxpy

In [2]:
import gpxpy
import gpxpy.gpx

import numpy as np
import pandas as pd

import folium

### Load your Polar Route GPX
- PyLoadRoutes
- PyDisplayRoutes

In [3]:
#Creating the Map where the route will be displayed

route_map = folium.Map(
    location=[47, 32],
    zoom_start=5,
    tiles='OpenStreetMap',
    width=924,
    height=600
)

In [4]:
def PyLoadRoutes(your_route_map,route_path):
    
    print(f"Loading the GPX files at: {route_path}")

    with open(route_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation
                })

    route_df = pd.DataFrame(route_info)

    return(route_df)

def PyDisplayRoutes(your_route_map,route_df,hex_color):

    coordinates = [tuple(x) for x in route_df[['latitude', 'longitude']].to_numpy()]
    folium.PolyLine(coordinates, weight=3,color=hex_color).add_to(route_map)
    folium.LayerControl().add_to(route_map)

    #if you want to display/save after adding every path separately:
    #display(route_map)
    #route_map.save('./output_map.html')

In [5]:
polar_route_df = PyLoadRoutes(route_map,'./Polar_Data/Polar_Route_.gpx')
PyDisplayRoutes(route_map,polar_route_df,'green')


#if you want to display/save the combined map to visualize all routes together:
display(route_map)
#route_map.save('./output_map.html')

Loading the GPX files at: ./Polar_Data/Polar_Route_.gpx


In [6]:
polar_route_df.head(5)

,latitude,longitude,elevation
0,52.291035,21.114608,0.0
1,52.291035,21.114608,0.0
2,52.291025,21.114608,0.0
3,52.291025,21.114608,0.0
4,52.291013,21.114614,0.0


In [7]:
#rows of the dataframe
polar_route_df.shape

(3793, 3)

### Load additional information from Polar
- PyLoadPolar
- PyPlotDistribution

In [34]:
def PyloadPolar(your_data_path):

    #read csv file with the data
    polar_data_df = pd.read_csv(your_data_path)

    #select only the columns we need
    polar_data_df = polar_data_df[['Time','HR (bpm)','Speed (km/h)']]

    polar_data_df = polar_data_df.fillna(0)

    return polar_data_df

# Remember to remove the first 2 rows from the exported csv file from Polar Flow
polar_data_df = PyloadPolar('./Polar_Data/Polar_Data_.csv'),

#tuple to pandas dataframe
polar_data_df = pd.DataFrame(polar_data_df[0])

polar_data_df.head(5)

,Time,HR (bpm),Speed (km/h)
0,00:00:00,0.0,0.0
1,00:00:01,0.0,0.0
2,00:00:02,0.0,0.3
3,00:00:03,0.0,0.3
4,00:00:04,0.0,0.6


In [35]:
#You should see matching number of rows for both dataframes
polar_data_df.shape

(3793, 3)

In [37]:
import plotly.figure_factory as ff
from scipy.stats import norm
import numpy as np

np.random.seed(123)

#defining a function to plot the distribution of the data, as it will be used multiple times

def PyPlotDistribution(data, title, x_axis, y_axis,group_descr):
    
    m, s = norm.fit(data)
    gaussian_data = np.random.normal(m, s, 10000)

    fig = ff.create_distplot(
        # [data, gaussian_data],
        # group_labels=[group_descr, f"Gaussian: m={m:.2f}, s={s:.2f}"],
        # curve_type="kde", show_rug=False
        [data],
        group_labels=[group_descr],
        curve_type="kde", show_rug=False
    )

    fig.update_layout(
        title={
            'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis=dict(
            title=x_axis
        ),
        yaxis=dict(
            title=y_axis
        ))

    fig.show()

    return(fig)

#Just display:
#PyPlotDistribution(polar_data_df['HR (bpm)'].tolist(), "Heart Rate Distribution", "Heart Rate (BPM)", "Y","HR Distribution")

#To display and save the figure:
PyPlotDistribution(polar_data_df['HR (bpm)'].tolist(), "Heart Rate Distribution", "Heart Rate (BPM)", "Y","HR Distribution")\
    .write_html("./Output_HR_Distrib.html")
                

In [38]:
PyPlotDistribution(polar_data_df['Speed (km/h)'].tolist(), "Speed Distribution", "Speed (km/h)", "Y","Speed Distribution")\
    .write_html("./Output_Speed_Distrib.html")
        

### Combining Both Datasets

In [39]:
#join both dataframes

combined_df = polar_route_df.join(polar_data_df)
combined_df.head(5)

,latitude,longitude,elevation,Time,HR (bpm),Speed (km/h)
0,52.291035,21.114608,0.0,00:00:00,0.0,0.0
1,52.291035,21.114608,0.0,00:00:01,0.0,0.0
2,52.291025,21.114608,0.0,00:00:02,0.0,0.3
3,52.291025,21.114608,0.0,00:00:03,0.0,0.3
4,52.291013,21.114614,0.0,00:00:04,0.0,0.6


In [40]:
combined_df.tail(5)

,latitude,longitude,elevation,Time,HR (bpm),Speed (km/h)
3788,52.256162,20.966554,0.0,01:03:08,114.0,8.8
3789,52.256162,20.966554,0.0,01:03:09,114.0,8.8
3790,52.256152,20.966486,0.0,01:03:10,114.0,7.7
3791,52.256153,20.966455,0.0,01:03:11,113.0,7.2
3792,52.256160,20.966419,0.0,01:03:12,113.0,7.2


In [52]:
#creating a conditional column on route_df based on elevation_diff:

combined_df['slope'] = np.where(combined_df['HR (bpm)'] > 135, 1, -1)

combined_df['slope_str'] = np.where(combined_df['HR (bpm)'] > 135, "Hammer Time", "Steady State")

combined_df['color_hex'] = np.where(combined_df['HR (bpm)'] > 135, "#E48403", "#119403")


#first ocurrance of Down in column slope_str
first_up_idx = combined_df.slope_str.ne('Hammer Time').idxmin()
first_down_idx = combined_df.slope_str.ne('Steady State').idxmin()


a=combined_df #.head(2000)

a["showlegend"] = False
showlegendIdx = a.columns.get_indexer(["showlegend"])[0]

a.iat[first_up_idx, showlegendIdx] = True
a.iat[first_down_idx, showlegendIdx] = True

a["showlegend"] = a["showlegend"].astype(object)


In [53]:
a.head(5)

,latitude,longitude,elevation,Time,HR (bpm),Speed (km/h),slope,slope_str,showlegend,color_hex
0,52.291035,21.114608,0.0,00:00:00,0.0,0.0,-1,Steady State,True,#119403
1,52.291035,21.114608,0.0,00:00:01,0.0,0.0,-1,Steady State,False,#119403
2,52.291025,21.114608,0.0,00:00:02,0.0,0.3,-1,Steady State,False,#119403
3,52.291025,21.114608,0.0,00:00:03,0.0,0.3,-1,Steady State,False,#119403
4,52.291013,21.114614,0.0,00:00:04,0.0,0.6,-1,Steady State,False,#119403


In [54]:
import numpy as np
import pandas as pd
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(
    [
        go.Scatter(
            #x = a.index[tn : tn + 2],
            x = a['Time'][tn : tn + 2],
            y = a['Speed (km/h)'][tn : tn + 2],
            #mode='lines+markers',
            mode='lines',
            # line_shape="hv",
            line_color=px.colors.qualitative.Plotly[a['slope'][tn]],
            name=a['slope_str'][tn],
            legendgroup= a['slope_str'][tn],
            showlegend= a['showlegend'][tn],
        )
        for tn in range(len(a))
    ]
)

fig.update_layout(legend_title_text='Climbing or Descending')

fig.update_layout(
    title={
        'text': "Speed Profile and Heart Rate",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(
        title="Time"
    ),
    yaxis=dict(
        title="Speed (km/h)"
    ))


fig.show()

In [65]:
route_map = folium.Map(
    location=[47, 32.5],
    zoom_start=5,
    tiles='OpenStreetMap',
    # width=924,
    # height=600
    width=800,
    height=450
)

In [84]:
#round the speed to integer
combined_df['Speed (km/h)'] = combined_df['Speed (km/h)'].round(0).astype(int)

In [86]:
speed_color

[(0,),
 (0,),
 (0,),
 (0,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (1,),
 (1,),
 (2,),
 (2,),
 (2,),
 (2,),
 (2,),
 (2,),
 (2,),
 (2,),
 (2,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (0,),
 (0,),
 (0,),
 (2,),
 (5,),
 (5,),
 (8,),
 (9,),
 (10,),
 (10,),
 (11,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (12,),
 (14,),
 (14,),
 (14,),
 (14,),
 (14,),
 (14,),
 (14,),
 (15,),
 (15,),
 (15,),
 (15,),
 (15,),
 (15,),
 (14,),
 (14,),
 (14,),
 (14,),
 (14,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (14,),
 (14,),
 (14,),
 (14,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (13,),
 (14,),
 (14,),
 (14,),
 (15,),
 (15,),
 (15,),
 (14,),
 (14,),
 (14,),
 (14,),
 (14,),
 (14,),
 (13,),
 (14,),
 (14,),
 (15,),
 (16,),
 (16,),
 (16,),
 (16,),
 (16,),
 (16,),
 (16,),
 (17,),
 (17,),
 (17,),
 (17,),
 (17,),
 (17,)

In [87]:
color_dict[speed_color]

TypeError: unhashable type: 'list'

In [89]:
#def PyLoadRoutes_Conditional(combined_df):
    
    

coordinates = [tuple(x) for x in combined_df[['latitude', 'longitude']].to_numpy()]
color_hex = [tuple(x) for x in combined_df[['color_hex']].to_numpy()]
speed_color = [tuple(x) for x in combined_df[['Speed (km/h)']].to_numpy()]

# folium.PolyLine(coordinates, weight=3, color=color_hex).add_to(route_map)
# folium.LayerControl().add_to(route_map)

folium.PolyLine(
        positions = coordinates, # tuple of coordinates 
        colors = color_hex #color_dict[speed_color] #color_hex # map each segment with the speed 
        #colormap =  colormap, # map each value with a color 
        ).add_to(route_map)

    #if you want to display/save after adding every path separately:
    #display(route_map)
    #route_map.save('./output_map.html')

    


TypeError: __init__() missing 1 required positional argument: 'locations'

In [61]:
route_map

### Example

In [67]:
coordinates = {'lat':[52.354, 52.081, 52.384], 'lng':[4.903, 5.099, 4.643]}
df = pd.DataFrame(data=coordinates, index=['Amsterdam', 'Utrecht', 'Haarlem'])
df

,lat,lng
Amsterdam,52.354,4.903
Utrecht,52.081,5.099
Haarlem,52.384,4.643


In [68]:
dict_speed = {}
cities_all = df.index.to_list()
 
for origin in df.index:
     dict_speed[origin] = {
         desti: np.random.uniform(3, 30) for desti in cities_all if origin != desti
     }
     cities_all.remove(origin)
 
dict_speed

{'Amsterdam': {'Utrecht': 21.749686241509075, 'Haarlem': 20.56589054089063},
 'Utrecht': {'Haarlem': 27.353463870502754},
 'Haarlem': {}}

In [69]:
list_colors = [
    "#00FF00",
    "#12FF00",
    "#24FF00",
    "#35FF00",
    "#47FF00",
    "#58FF00",
    "#6AFF00",
    "#7CFF00",
    "#8DFF00",
    "#9FFF00",
    "#B0FF00",
    "#C2FF00",
    "#D4FF00",
    "#E5FF00",
    "#F7FF00",
    "#FFF600",
    "#FFE400",
    "#FFD300",
    "#FFC100",
    "#FFAF00",
    "#FF9E00",
    "#FF8C00",
    "#FF7B00",
    "#FF6900",
    "#FF5700",
    "#FF4600",
    "#FF3400",
    "#FF2300",
    "#FF1100",
    "#FF0000",
]
color_dict = {i: list_colors[i] for i in range(len(list_colors))}

In [81]:
len(list_colors)

30

In [76]:
color_dict[20]

'#FF9E00'

In [73]:
type(color_dict)

dict

In [71]:
df

,lat,lng
Amsterdam,52.354,4.903
Utrecht,52.081,5.099
Haarlem,52.384,4.643


In [90]:
my_map = folium.Map(
    location=[np.mean(df["lat"].values), np.mean(df["lng"].values)],
    zoom_start=7,
    tiles="Stamen Terrain",
)

# for city, row in df.iterrows():
#     folium.Marker([row["lat"], row["lng"]], tooltip=city).add_to(my_map)

for origin, _ in dict_speed.items():
    for desti, speed in _.items():
        folium.PolyLine(
            (df.loc[origin].values, df.loc[desti].values),
            color=color_dict[round(speed)],
        ).add_to(my_map)

my_map